# Approach-1) TfIDF+ Classical ML

In [ ]:
# !pip install xgboost transformers --quiet


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_excel("/content/bank.xlsx")
df

,User,Transaction,Tag
0,User1,MPS/TRUFFLES /202303261700/034587/Bangalore,Food
1,User1,MPS/TACO BELL /202304012247/108300/BANGALORE,Food
2,User2,POS XXXXXXXXXXXX0001 APOLLO PHARMACY,Medical
3,User2,BIL/ONL/000471093694/1MG Techno/X7ZRUSVLURFQZO,Medical
4,User1,POS XXXXXXXXXXXX1111 DECATHLON SPORTS,Shopping
5,User2,POS XXXXXXXXXXXX1111 IKEA INDIA PVT L,Shopping
6,User3,POS XXXXXXXXXXXX1111 WWW AMAZON IN,Shopping
7,User4,ME DC SI XXXXXXXXXXXX1111 SPOTIFY SI,Subscription
8,User5,POS/NETFLIX/1140920002/100623/17:25,Subscription
9,User5,POS XXXXXXXXXXXX1110 MAKEMYTRIP INDIA,Travel


In [ ]:
import re
def preprocess(txt):
    txt=txt.replace("/"," ")
    txt=re.sub("[0-9]","",txt)
    # txt=re.sub("^[a-zA-Z]X+","",txt)

    txt=" ".join([re.sub("^[a-zA-Z]X+","",w)for w in txt.split()])

    txt=re.sub("\s+"," ",txt)
    txt=re.sub("[^a-zA-Z0-9\s]","",txt)

    return txt

preprocess("POS XXXXXXXXXXXX1111 DECATHLON SPORTS")

'POS DECATHLON SPORTS'

In [ ]:
df["cleaned"]=df["Transaction"].apply(preprocess)
df

,User,Transaction,Tag,cleaned
0,User1,MPS/TRUFFLES /202303261700/034587/Bangalore,Food,MPS TRUFFLES Bangalore
1,User1,MPS/TACO BELL /202304012247/108300/BANGALORE,Food,MPS TACO BELL BANGALORE
2,User2,POS XXXXXXXXXXXX0001 APOLLO PHARMACY,Medical,POS APOLLO PHARMACY
3,User2,BIL/ONL/000471093694/1MG Techno/X7ZRUSVLURFQZO,Medical,BIL ONL MG Techno XZRUSVLURFQZO
4,User1,POS XXXXXXXXXXXX1111 DECATHLON SPORTS,Shopping,POS DECATHLON SPORTS
5,User2,POS XXXXXXXXXXXX1111 IKEA INDIA PVT L,Shopping,POS IKEA INDIA PVT L
6,User3,POS XXXXXXXXXXXX1111 WWW AMAZON IN,Shopping,POS WWW AMAZON IN
7,User4,ME DC SI XXXXXXXXXXXX1111 SPOTIFY SI,Subscription,ME DC SI SPOTIFY SI
8,User5,POS/NETFLIX/1140920002/100623/17:25,Subscription,POS NETFLIX
9,User5,POS XXXXXXXXXXXX1110 MAKEMYTRIP INDIA,Travel,POS MAKEMYTRIP INDIA


In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [ ]:
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(df["Tag"])
df["y_encoded"]=encoded_tags

In [ ]:
X = df['cleaned']
y = df['y_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
X_test

1    MPS TACO BELL BANGALORE
6          POS WWW AMAZON IN
8               POS NETFLIX 
9       POS MAKEMYTRIP INDIA
Name: cleaned, dtype: object

In [ ]:
vectorizer = TfidfVectorizer()

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [ ]:
import pickle

pickle.dump(vectorizer, open("tfidf.pkl", "wb"))


In [ ]:
X_test_vectorized

<4x31 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB

model = RandomForestClassifier(n_estimators=200)
model.fit(X_train_vectorized.toarray() , y_train)
y_pred = svm_model.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       0.33      1.00      0.50         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.50         4
   macro avg       0.33      0.50      0.38         4
weighted avg       0.33      0.50      0.38         4



In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train_vectorized.toarray() , y_train)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       0.33      1.00      0.50         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.50         4
   macro avg       0.33      0.50      0.38         4
weighted avg       0.33      0.50      0.38         4



In [ ]:
label_dict={
    0:"Food",
    1:"Medical",
    2:"Shopping",
    3:"Subscription",
    4:"Travel",
}

In [ ]:

def recommend_tags(transaction):
    transaction=preprocess(transaction)
    transaction_features = vectorizer.transform([transaction])
    probs = model.predict_proba(transaction_features)[0]
    predicted_tags={i:p for i,p in enumerate(probs)}
    predicted_tags=sorted(predicted_tags.items(),key= lambda x: x[1] , reverse=True)
    # recommended_tags = label_encoder.inverse_transform(predicted_tags)
    for k,v in predicted_tags[:3]:
      print(label_dict[k],v)

# Example usage:
transaction = "POS XXXXXXXXXXXX1111 DECATHLON SPORTS"
recommended_tags = recommend_tags(transaction)



Shopping 0.825
Medical 0.135
Food 0.015


In [ ]:
sklearn.__version__

'1.2.2'

In [ ]:
import pickle
with open("rf_model.pkl","wb") as f:
  pickle.dump(model,f)

In [ ]:
with open("rf_model.pkl","rb") as f:
  m=pickle.load(f)

# 2) Bert + RF


In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [ ]:
X = df['cleaned']
y = df['y_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
y_test

1    0
6    2
8    3
9    4
Name: y_encoded, dtype: int64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def encode_text(text):
    input_ids = []
    attention_masks = []

    for statement in text:
        encoded_dict = tokenizer.encode_plus(
                            statement,
                            add_special_tokens=True,
                            max_length=64,
                            pad_to_max_length=True,
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks


In [ ]:
# Encode the training and testing data
train_input_ids, train_attention_masks = encode_text(X_train)
test_input_ids, test_attention_masks = encode_text(X_test)



In [ ]:
# Obtain the BERT embeddings for the training and testing data
with torch.no_grad():
    train_embeddings = model(input_ids=train_input_ids, attention_mask=train_attention_masks)[0][:, 0, :]
    test_embeddings = model(input_ids=test_input_ids, attention_mask=test_attention_masks)[0][:, 0, :]

rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(train_embeddings, y_train)

In [ ]:
y_pred = rf_classifier.predict(test_embeddings)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.25         4
   macro avg       0.20      0.20      0.20         4
weighted avg       0.25      0.25      0.25         4

